In [7]:
import os
import shutil
import pandas as pd

## Перед началом, создай копию своего изначального датасета,который ты почистил

Твоя папка будет состоять из следующих файлов:
- Изначальный датасет
- 1_custom_tensorflow_dataset
- dataset
     |- 2_custom_dataset
     
     
ты создаешь папку датасет и добавляешь в него 2_custom_dataset. 1_custom_tensorflow_dataset ты запускаешь первым, после чего переходишь к 2_custom_dataset и работаешь в нем

In [2]:
# Меняем слеши для Windows
def make_path_right(path):
    path = path.replace('\\','/')
    return path

In [5]:
# Папка с изначальным датасетом, в нашем случае с датасетом, где оставлены только поперечные мрт
root_dir = './SCLEROSIS21_172/'
# Создаем список с путями до всех папок, в которых есть файлы DICOM
# при этом иcпользуем не os.listdir(), а os.scandir(), потому что пути до папок слишком длинные
dicom_dirs = []
for scanner_dir in os.scandir(root_dir):
    if scanner_dir.is_dir():
        for study_dir in os.scandir(scanner_dir):
            if study_dir.is_dir():
                for series_dir in os.scandir(study_dir):
                    if series_dir.is_dir():
                        if any(file.name.endswith('.dcm') for file in os.scandir(series_dir)):
                            directory = make_path_right(series_dir.path)
                            dicom_dirs.append(directory)


In [6]:
dicom_dirs[:5]

['./SCLEROSIS21_172/EXCELMR1_GB1_anon/1.2.643.5.1.13.13.12.2.77.8252.01101314101209110010040306071511/1.2.643.5.1.13.13.12.2.77.8252.00000012111108071003041505040813',
 './SCLEROSIS21_172/EXCELMR1_GB1_anon/1.2.643.5.1.13.13.12.2.77.8252.01101314101209110010040306071511/1.2.643.5.1.13.13.12.2.77.8252.00041407070411100306000901020002',
 './SCLEROSIS21_172/EXCELMR1_GB1_anon/1.2.643.5.1.13.13.12.2.77.8252.01101314101209110010040306071511/1.2.643.5.1.13.13.12.2.77.8252.01001503121507140008050007091013',
 './SCLEROSIS21_172/EXCELMR1_GB1_anon/1.2.643.5.1.13.13.12.2.77.8252.01101314101209110010040306071511/1.2.643.5.1.13.13.12.2.77.8252.04031105030809060511070011111100',
 './SCLEROSIS21_172/EXCELMR1_GB1_anon/1.2.643.5.1.13.13.12.2.77.8252.01101314101209110010040306071511/1.2.643.5.1.13.13.12.2.77.8252.04080812100210010709030900111215']

In [56]:

# Целевая директория, в которую мы будем переносить наши папки с DICOM и в которой в последствии мы создадим
# новый ноутбук в котором продолдим работу, потому что папки открываются только в той же директории, в которой
# и сам ноутбук. Именно в это директорию нужно положить ноутбук 2_custom_dataset
target_dir = "./dataset"

In [57]:
# Проходимся по списку с директориями и переносим их в нашу новую директорию 
for dir_path in dicom_dirs:
    shutil.move(dir_path, target_dir)

### Создаем pandas dataframe с разметкой

Здесь мы делаем так, чтобы картинки и таргет были синхронизированы, после запуска всех ячеек переходи в папку 'dataset', в которой должен лежать ноутбук 2_custom_dataset

In [39]:
result = pd.DataFrame(columns=['folder', 'study_uid'])
for path in dicom_dirs:
    path = path.split('/')
    new_row = pd.DataFrame({'folder':[path[4]], 'study_uid':[path[3]]})
    result = pd.concat([result, new_row], ignore_index=True)

In [36]:
result

,folder,study_uid
0,1.2.643.5.1.13.13.12.2.77.8252.000000121111080...,1.2.643.5.1.13.13.12.2.77.8252.011013141012091...
1,1.2.643.5.1.13.13.12.2.77.8252.000414070704111...,1.2.643.5.1.13.13.12.2.77.8252.011013141012091...
2,1.2.643.5.1.13.13.12.2.77.8252.010015031215071...,1.2.643.5.1.13.13.12.2.77.8252.011013141012091...
3,1.2.643.5.1.13.13.12.2.77.8252.040311050308090...,1.2.643.5.1.13.13.12.2.77.8252.011013141012091...
4,1.2.643.5.1.13.13.12.2.77.8252.040808121002100...,1.2.643.5.1.13.13.12.2.77.8252.011013141012091...


In [33]:
labels = pd.read_excel('labels.xlsx')

In [34]:
labels

,study_uid,sclerosis
0,1.2.643.5.1.13.13.12.2.77.8252.011411090410101...,0
1,1.2.643.5.1.13.13.12.2.77.8252.131500041112120...,1
2,1.2.643.5.1.13.13.12.2.77.8252.010305060009140...,0
3,1.2.643.5.1.13.13.12.2.77.8252.071406031309010...,1
4,1.2.643.5.1.13.13.12.2.77.8252.121511041410151...,0
...,...,...
167,1.2.643.5.1.13.13.12.2.77.8252.111211060701140...,1
168,1.2.643.5.1.13.13.12.2.77.8252.100815141103051...,0
169,1.2.643.5.1.13.13.12.2.77.8252.060804010900090...,1
170,1.2.643.5.1.13.13.12.2.77.8252.030410101303070...,0


In [40]:
merged = pd.merge(result, labels, on='study_uid', how='left')

In [47]:
merged['sclerosis'].value_counts()

1    956
0    869
Name: sclerosis, dtype: int64

In [51]:
merged.drop('study_uid',inplace=True, axis=1)

In [54]:
merged.to_csv('dataset.csv', index=False)